In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath(".."))
from annnet.core.graph import AnnNet

### Composite indexing

In [ ]:

G = AnnNet()

G.set_vertex_key("name", "sex")

m = G.get_or_create_vertex_by_attrs(name="a", sex="man")
w = G.get_or_create_vertex_by_attrs(name="a", sex="woman")

print(m, w, m == w)   # expect two different ids, False



In [ ]:
G.set_vertex_key("name")


In [ ]:
m = G.get_or_create_vertex_by_attrs(name="a", sex="man")
w = G.get_or_create_vertex_by_attrs(name="a", sex="woman")
print(m, w, m == w)   # same id, True

### Flexible directionality

In [ ]:
G = AnnNet()
G.set_vertex_key("name","sex")

u = G.get_or_create_vertex_by_attrs(name="a", sex="man")
v = G.get_or_create_vertex_by_attrs(name="a", sex="woman")

# Helper to read signs
def signs(G, eid):
    s,t,_ = G.edge_definitions[eid]
    col = G.edge_to_idx[eid]
    si = G.entity_to_idx[s]; ti = G.entity_to_idx[t]
    M = G._matrix
    return M.get((si,col),0), M.get((ti,col),0)

# Edge-scope policy
e = G.add_edge(u, v,
    flexible={"var":"capacity", "threshold":0.7, "scope":"edge", "above":"s->t", "tie": "undirected"},
    capacity=0.5, weight=1.0)

print("init:", signs(G,e))            # expect (-1, +1)
G.set_edge_attrs(e, capacity=0.9)
print("after:", signs(G,e))           # expect (+1, -1)




In [ ]:
# Vertex-scope policy
G2 = AnnNet()
G2.set_vertex_key("name","sex")
s = G2.get_or_create_vertex_by_attrs(name="a", sex="man")
t = G2.get_or_create_vertex_by_attrs(name="a", sex="woman")
G2.set_vertex_attrs(s, temp=10.0)
G2.set_vertex_attrs(t, temp=20.0)

e2 = G2.add_edge(s, t,
    flexible={"var":"temp","threshold":0.0,"scope":"vertex","above":"s->t","tie":"undirected"},
    weight=1.0)

print("xs<xt:", signs(G2,e2))         # (-1, +1)
G2.set_vertex_attrs(s, temp=25.0)
print("xs>xt:", signs(G2,e2))         # (+1, -1)
G2.set_vertex_attrs(t, temp=25.0)
print("tie  :", signs(G2,e2))         # since "tie":"undirected", it becomes: (+1, +1)

### Kivela Multilayers

In [ ]:
# ---------- build a tiny Kivela multilayer graph ----------

G = AnnNet(directed=True)

# 1) Define aspects and elementary layers
G.set_aspects(
    aspects=["time"],
    elem_layers={"time": ["t1", "t2"]},
)

# 2) Add vertices in the base graph
for u in ["A", "B", "C"]:
    G.add_vertex(u, name = f"named {u}")

# 3) Declare vertex-layer presence (V_M)
for u in ["A", "B", "C"]:
    G.add_presence(u, ("t1",))
    G.add_presence(u, ("t2",))

# 4) Add Kivela edges (all go through incidence under the hood)

# intra edges in t1
G.add_intra_edge_nl("A", "B", ("t1",), weight=1.0)
G.add_intra_edge_nl("B", "C", ("t1",), weight=1.0)

# intra edges in t2
G.add_intra_edge_nl("A", "C", ("t2",), weight=2.0)

# inter-layer edge between A@t1 and B@t2
G.add_inter_edge_nl("A", ("t1",), "B", ("t2",), weight=0.5)

# coupling edges A: t1 <-> t2 and B: t1 <-> t2
G.add_coupling_edge_nl("A", ("t1",), ("t2",), weight=1.0)
G.add_coupling_edge_nl("B", ("t1",), ("t2",), weight=1.0)

# ---------- annotate layers and vertex-layer pairs ----------

# Elementary layer attributes, stored in G.layer_attributes
G.set_elementary_layer_attrs("time", "t1", order=1, name="early")
G.set_elementary_layer_attrs("time", "t2", order=2, name="late")

# vertex-layer attributes, stored in _vertex_layer_attrs
G.set_vertex_layer_attrs("A", ("t1",), activity=0.2, color="blue")
G.set_vertex_layer_attrs("A", ("t2",), activity=0.9, color="red")
G.set_vertex_layer_attrs("B", ("t1",), activity=0.5)
G.set_vertex_layer_attrs("B", ("t2",), activity=0.7)

print("vertex-layer attrs A@t1:", G.get_vertex_layer_attrs("A", ("t1",)))
print("vertex-layer attrs A@t2:", G.get_vertex_layer_attrs("A", ("t2",)))
print()

# ---------- supra structures ----------

A_supra = G.supra_adjacency()
print("Supra adjacency shape:", A_supra.shape)
print("Supra adjacency matrix:\n", A_supra.toarray())
print()

L_comb = G.supra_laplacian(kind="comb")
L_norm = G.supra_laplacian(kind="norm")
print("Combinatorial Laplacian:\n", L_comb.toarray())
print()
print("Normalized Laplacian:\n", L_norm.toarray())
print()

print("vertex-layer index (row -> (vertex, layer_tuple)):")
for i, (u, aa) in enumerate(G._row_to_nl):
    print(f"  {i}: {u} @ {aa}")
print()

deg = G.supra_degree()
print("Supra degrees per vertex-layer row:", deg)
print()

print("Participation coefficient per vertex:", G.participation_coefficient())
print("Versatility per vertex:", G.versatility())
print()

print("Layer attribute table (Polars):")
print(G.layer_attributes)


In [ ]:
# ---------- Kivela layer algebra & slice operations ----------

aa_t1 = ("t1",)
aa_t2 = ("t2",)

print("\n=== Kivela layer vertex/edge sets ===")
print("Vertices in layer t1:", G.layer_vertex_set(aa_t1))
print("Vertices in layer t2:", G.layer_vertex_set(aa_t2))

print("Intra edges in t1:", G.layer_edge_set(aa_t1))
print("Intra edges in t2:", G.layer_edge_set(aa_t2))

print("Edges touching t1 (intra+inter+coupling):",
      G.layer_edge_set(aa_t1, include_inter=True, include_coupling=True))
print("Edges touching t2 (intra+inter+coupling):",
      G.layer_edge_set(aa_t2, include_inter=True, include_coupling=True))

# ---------- pure Kivela layer algebra (set-based) ----------

print("\n=== Kivela layer algebra (set view) ===")
res_union = G.layer_union([aa_t1, aa_t2])
print("Union vertices (t1 ∪ t2):", res_union["vertices"])
print("Union edges   (t1 ∪ t2):", res_union["edges"])

res_inter = G.layer_intersection([aa_t1, aa_t2])
print("Intersection vertices (t1 ∩ t2):", res_inter["vertices"])
print("Intersection edges   (t1 ∩ t2):", res_inter["edges"])

res_diff = G.layer_difference(aa_t1, aa_t2)
print("Difference vertices (t1 \\ t2):", res_diff["vertices"])
print("Difference edges   (t1 \\ t2):", res_diff["edges"])

# ---------- Kivela → slice bridge ----------

print("\n=== Slice creation from Kivela layers ===")

# single-layer slices
sid_t1 = G.create_slice_from_layer("L_t1", aa_t1)
sid_t2 = G.create_slice_from_layer("L_t2", aa_t2)

print("All slices (incl. default):", G.list_slices(include_default=True))
print("Slice L_t1 vertices:", G.get_slice_vertices(sid_t1))
print("Slice L_t1 edges   :", G.get_slice_edges(sid_t1))
print("Slice L_t2 vertices:", G.get_slice_vertices(sid_t2))
print("Slice L_t2 edges   :", G.get_slice_edges(sid_t2))

# union / intersection / difference slices
sid_union = G.create_slice_from_layer_union("L_union_t1_t2", [aa_t1, aa_t2])
sid_inter = G.create_slice_from_layer_intersection("L_inter_t1_t2", [aa_t1, aa_t2])
sid_diff  = G.create_slice_from_layer_difference("L_diff_t1_t2", aa_t1, aa_t2)

print("Slice L_union_t1_t2 vertices:", G.get_slice_vertices(sid_union))
print("Slice L_union_t1_t2 edges   :", G.get_slice_edges(sid_union))

print("Slice L_inter_t1_t2 vertices:", G.get_slice_vertices(sid_inter))
print("Slice L_inter_t1_t2 edges   :", G.get_slice_edges(sid_inter))

print("Slice L_diff_t1_t2 vertices:", G.get_slice_vertices(sid_diff))
print("Slice L_diff_t1_t2 edges   :", G.get_slice_edges(sid_diff))

# ---------- Subgraphs from Kivela layers ----------

print("\n=== Subgraphs from Kivela layers ===")

Gs_t1 = G.subgraph_from_layer_tuple(aa_t1)
Gs_t2 = G.subgraph_from_layer_tuple(aa_t2)
Gs_union = G.subgraph_from_layer_union([aa_t1, aa_t2])
Gs_diff = G.subgraph_from_layer_difference(aa_t1, aa_t2)

print("Subgraph t1 vertices:", set(Gs_t1.entity_types.keys()))
print("Subgraph t1 edges   :", set(Gs_t1.edge_definitions.keys()))

print("Subgraph t2 vertices:", set(Gs_t2.entity_types.keys()))
print("Subgraph t2 edges   :", set(Gs_t2.edge_definitions.keys()))

print("Subgraph union(t1,t2) vertices:", set(Gs_union.entity_types.keys()))
print("Subgraph union(t1,t2) edges   :", set(Gs_union.edge_definitions.keys()))

print("Subgraph t1\\t2 vertices:", set(Gs_diff.entity_types.keys()))
print("Subgraph t1\\t2 edges   :", set(Gs_diff.edge_definitions.keys()))

# ---------- Optional: test LayerManager if exposed as G.layers ----------

if hasattr(G, "layers"):
    print("\n=== LayerManager high-level API ===")
    print("All layer tuples:", G.layers.layer_tuples())
    print("vertex_set(t1):", G.layers.vertex_set(aa_t1))
    print("edge_set(t1):", G.layers.edge_set(aa_t1))

    lm_union = G.layers.union([aa_t1, aa_t2])
    print("LayerManager union vertices:", lm_union["vertices"])
    print("LayerManager union edges   :", lm_union["edges"])

    lm_sid_t1 = G.layers.to_slice(aa_t1, slice_id="LM_t1")
    print("LM_t1 slice vertices:", G.get_slice_vertices(lm_sid_t1))
    print("LM_t1 slice edges   :", G.get_slice_edges(lm_sid_t1))

    Gs_lm = G.layers.subgraph(aa_t1)
    print("LayerManager subgraph(t1) vertices:", set(Gs_lm.entity_types.keys()))
    print("LayerManager subgraph(t1) edges   :", set(Gs_lm.edge_definitions.keys()))


### graphtool

In [ ]:
import graph_tool.all as gt

In [ ]:
import os
import sys

ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if ROOT not in sys.path:
    sys.path.insert(0, ROOT)

from annnet.adapters import graphtool_adapter as gtt

In [ ]:
gtG, manifest = gtt.to_graphtool(G)

# reconstruct an AnnNet graph:
G2 = gtt.from_graphtool(gtG, manifest)


In [ ]:
gtG

In [ ]:
pos = gt.sfdp_layout(gtG)

# Draw with vertex ids as labels
gt.graph_draw(
    gtG,
    pos=pos,
    vertex_text=gtG.vp["id"],
    vertex_font_size=10,
    output_size=(300,300),
)

In [ ]:
# choose a layer tuple
aa = ("t1",)  # for single-aspect "time"

# subgraph of that layer
G_t1 = G.subgraph_from_layer_tuple(aa)

# project and draw
gtG_t1, man_t1 = gtt.to_graphtool(G_t1)
pos_t1 = gt.sfdp_layout(gtG_t1)
gt.graph_draw(
    gtG_t1,
    pos=pos_t1,
    vertex_text=gtG_t1.vp["id"],
    vertex_font_size=10,
    output_size=(600, 600),
)

In [ ]:
manifest

### gt lazy proxies adapter

In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath(".."))
from annnet.core.graph import AnnNet

In [ ]:
g = AnnNet()

In [ ]:
g.add_vertex("A", age = 5)

In [ ]:
g.add_vertex("B", age = 7.9)

In [ ]:
g.vertex_attributes

In [ ]:
g.add_edge("A", "B", n = 78)

In [ ]:
g.add_edge("A", "B", n = 67)

In [ ]:
g.edge_attributes

In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath(".."))
from annnet.core.graph import AnnNet

In [ ]:
def build_simple():
    G = AnnNet()
    G.add_vertex("a")
    G.add_vertex("b")
    G.add_edge("a", "b", weight=5.0)
    return G

# Basic call
G = build_simple()
dist = G.gt.topology.shortest_distance(G, source="a", target="b", weights="weight")
print("distance:", float(dist))   # expected 5.0

In [ ]:
# Directed
G = AnnNet(directed=True)
G.add_vertex("a")
G.add_vertex("b")
G.add_edge("a", "b", weight=2.0)

d1 = G.gt.topology.shortest_distance(G, source="a", target="b", weights="weight")
print("directed distance:", float(d1))  # expected 2.0

try:
    d2 = G.gt.topology.shortest_distance(G, source="b", target="a", weights="weight")
    print("reverse directed distance:", d2)
except Exception as e:
    print("reverse failed:", type(e).__name__)


In [ ]:
G = build_simple()

# AnnNet-tool version: must be `gtG.ep['weight']`
gtG = G.gt.backend()

# direct-graph-tool call (no proxy)
from graph_tool import topology
d_ref = topology.shortest_distance(gtG,
                                   source=gtG.vertex(0),
                                   target=gtG.vertex(1),
                                   weights=gtG.ep["weight"])

d_proxy = G.gt.topology.shortest_distance(G, source="a", target="b", weights="weight")

print("direct vs proxy:", float(d_ref), float(d_proxy))


In [ ]:
G = AnnNet()
G.add_vertex("x")
G.add_vertex("y")
G.add_edge("x", "y", weight=10.0)
G.add_edge("x", "y", weight=1.0)    # smallest weight should dominate shortest paths

d = G.gt.topology.shortest_distance(G, source="x", target="y", weights="weight")
print("parallel edges distance:", float(d))  # expected 1.0


In [ ]:
G = AnnNet()
G.add_vertex("a")
G.add_vertex("b")
G.add_vertex("c")
G.add_edge("a", "b")
G.add_edge("b", "c")

comp = G.gt.topology.label_components(G)
vp = comp[0]  # vertex property map

print("component of a:", int(vp[G.gt.backend().vertex(0)]))
print("component of c:", int(vp[G.gt.backend().vertex(2)]))


In [ ]:
G = AnnNet()
G.add_vertex("a")
G.add_vertex("b")
G.add_vertex("c")
G.add_edge("a", "b")
G.add_edge("b", "c")

vc, ec = G.gt.centrality.betweenness(G)

print("betweenness(a) =", float(vc[G.gt.backend().vertex(0)]))
print("betweenness(b) =", float(vc[G.gt.backend().vertex(1)]))
print("betweenness(c) =", float(vc[G.gt.backend().vertex(2)]))


In [ ]:
dmap = G.gt.topology.shortest_distance(G, source="a", weights="weight")
print("distance map entry b:", float(dmap[G.gt.backend().vertex(1)]))


In [ ]:
G = build_simple()

try:
    G.gt.topology.shortest_distance(G, source="zzz", target="a", weights="weight")
except Exception as e:
    print("Caught error:", type(e).__name__, str(e))


In [ ]:
g= G

In [ ]:
g.V

In [ ]:
g.vertex_attributes

In [ ]:
g.add_vertex("v", name = 67)

In [ ]:
g.vertex_attributes

In [ ]:
g.set_vertex_attrs("v", a = 8)

In [ ]:
g.vertex_attributes

In [ ]:
g.E

In [ ]:
g.set_edge_attrs("edge_0", age = 78)

In [ ]:
g.edge_attributes

In [ ]:
g.set_vertex_attrs("v", name = 8888)

In [ ]:
g.vertex_attributes

In [ ]:
g.set_vertex_attrs("tt")

In [ ]:
g.vertex_attributes

### CX2 adapter

In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath(".."))
from annnet.core.graph import AnnNet
from annnet.adapters import cx2_adapter as cx

In [ ]:
from annnet.io import io_annnet as ia



In [ ]:
G = ia.read("ppi.annnet")

In [ ]:
G.vertex_attributes

In [ ]:
sk = cx.to_cx2(G)

ex = cx.to_cx2(G, hyperedges= "expand")

rf = cx.to_cx2(G, hyperedges= "reify")

In [ ]:
import json

# Assuming 'cx2_data' is the variable holding your list of dictionaries
output_file = "sk.cx2"

with open(output_file, "w") as f:
    json.dump(sk, f)

print(f"Saved to {output_file}")

In [ ]:
import json

# Assuming 'cx2_data' is the variable holding your list of dictionaries
output_file = "ex.cx2"

with open(output_file, "w") as f:
    json.dump(ex, f)

print(f"Saved to {output_file}")

In [ ]:
import json

# Assuming 'cx2_data' is the variable holding your list of dictionaries
output_file = "rf.cx2"

with open(output_file, "w") as f:
    json.dump(rf, f)

print(f"Saved to {output_file}")

In [ ]:
def build_simple():
    G = AnnNet()
    G.add_vertex("a")
    G.add_vertex("b")
    G.add_edge("a", "b", weight=5.0)
    G.add_hyperedge(members=["A","C","D"], weight=1.0, tag="complex")
    G.add_hyperedge(head=["A","B"], tail=["C","D"], weight=1.0, reaction="A+B->C+D")
    return G

# Basic call
Gs = build_simple()
ts = cx.to_cx2(Gs, hyperedges= "reify")

In [ ]:
import json

# Assuming 'cx2_data' is the variable holding your list of dictionaries
output_file = "ts.cx2"

with open(output_file, "w") as f:
    json.dump(ts, f)

print(f"Saved to {output_file}")

In [ ]:
tsg= cx.from_cx2('ts.cx2')

tsg.edges_view()

In [ ]:
skg= cx.from_cx2('sk.cx2')


In [ ]:
exg = cx.from_cx2('ex.cx2')

In [ ]:
rfg = cx.from_cx2("rf.cx2")

In [ ]:
skg.vertex_attributes

## AnnNet explore

In [ ]:
g = AnnNet()

In [ ]:
g.add_edge("a", "b",  edge_id= "aa12", as_entity=True, name = 12)

In [ ]:
g.add_edge("a", "c",  edge_id= "a13", as_entity=True, age = "rezf")

In [ ]:
g.V

In [ ]:
g.add_edge("aa12", "c",  edge_id= 13, as_entity=True, a = 67)

In [ ]:
g.E

In [ ]:
g.add_edge("aa12", "a13", name = "UIYG", a = 6.9)

In [ ]:
g.E

In [ ]:
g.edge_attributes

In [ ]:
g.add_hyperedge(members= ["a", "b", "aa12"])

In [ ]:
g.set_edge_attrs("edge_1", name = "233", zefzf = 1)

In [ ]:
g.edge_attributes

In [ ]:
g.edge_definitions

In [ ]:
g.edges_view()

In [ ]:
g._matrix

## Benchmark

In [ ]:
# edge entites
from annnet.core.graph import AnnNet
from benchmarks.harness.metrics import measure
import tracemalloc


def _measure_mem(fn):
    tracemalloc.start()
    before = tracemalloc.take_snapshot()
    fn()
    after = tracemalloc.take_snapshot()
    tracemalloc.stop()

    stats = after.compare_to(before, "filename")
    peak_bytes = sum(stat.size_diff for stat in stats)
    return peak_bytes


def run(scale):
    results = {}
    G = AnnNet(directed=True)

    # ------------------------------------------------------------
    # 0) Base graph construction (baseline cost)
    # ------------------------------------------------------------
    with measure() as m_vertices:
        mem_vertices = _measure_mem(
            lambda: G.add_vertices_bulk(
                ({"vertex_id": f"v{i}"} for i in range(scale.vertices)),
                slice="base",
            )
        )

    with measure() as m_edges:
        mem_edges = _measure_mem(
            lambda: G.add_edges_bulk(
                {
                    "source": f"v{i % scale.vertices}",
                    "target": f"v{(i + 1) % scale.vertices}",
                    "weight": 1.0,
                }
                for i in range(scale.edges)
            )
        )

    base_edges = list(G.edges())

    results["base_vertices"] = {
        "count": scale.vertices,
        "wall_time_s": m_vertices["wall_time_s"],
        "bytes": mem_vertices,
        "bytes_per_item": mem_vertices / max(1, scale.vertices),
    }

    results["base_edges"] = {
        "count": scale.edges,
        "wall_time_s": m_edges["wall_time_s"],
        "bytes": mem_edges,
        "bytes_per_item": mem_edges / max(1, scale.edges),
    }

    # ------------------------------------------------------------
    # 1) Node -> Edge edge creation (no reification)
    # ------------------------------------------------------------
    node_edge_ids = []

    def _node_edge_create():
        for i, eid in enumerate(base_edges):
            u = f"v{i % scale.vertices}"
            node_edge_ids.append(
                G.add_edge(u, eid, weight=1.0, as_entity=False)
            )

    with measure() as m_node_edge_create:
        mem_node_edge_create = _measure_mem(_node_edge_create)

    results["node_to_edge_create"] = {
        "count": len(node_edge_ids),
        "wall_time_s": m_node_edge_create["wall_time_s"],
        "bytes": mem_node_edge_create,
        "bytes_per_item": mem_node_edge_create / max(1, len(node_edge_ids)),
    }

    # ------------------------------------------------------------
    # 2) Edge reification (bulk)
    # ------------------------------------------------------------
    with measure() as m_reify:
        mem_reify = _measure_mem(
            lambda: G.add_edge_entities_bulk(node_edge_ids, slice="edge_entities")
        )

    results["edge_reification_bulk"] = {
        "count": len(node_edge_ids),
        "wall_time_s": m_reify["wall_time_s"],
        "bytes": mem_reify,
        "bytes_per_item": mem_reify / max(1, len(node_edge_ids)),
    }

    # ------------------------------------------------------------
    # 3) Edge -> Edge creation + reification
    # ------------------------------------------------------------
    edge_entities = [
        e for e in node_edge_ids
        if e in G.entity_types and G.entity_types[e] == "edge"
    ]

    ee_ids = []

    def _edge_edge_create():
        for i in range(len(edge_entities) - 1):
            ee_ids.append(
                G.add_edge(
                    edge_entities[i],
                    edge_entities[i + 1],
                    weight=1.0,
                    as_entity=False,
                )
            )

    with measure() as m_edge_edge_create:
        mem_edge_edge_create = _measure_mem(_edge_edge_create)

    with measure() as m_edge_edge_reify:
        mem_edge_edge_reify = _measure_mem(
            lambda: G.add_edge_entities_bulk(ee_ids, slice="edge_edge_entities")
        )

    results["edge_to_edge_create"] = {
        "count": len(ee_ids),
        "wall_time_s": m_edge_edge_create["wall_time_s"],
        "bytes": mem_edge_edge_create,
        "bytes_per_item": mem_edge_edge_create / max(1, len(ee_ids)),
    }

    results["edge_to_edge_reify"] = {
        "count": len(ee_ids),
        "wall_time_s": m_edge_edge_reify["wall_time_s"],
        "bytes": mem_edge_edge_reify,
        "bytes_per_item": mem_edge_edge_reify / max(1, len(ee_ids)),
    }

    # ------------------------------------------------------------
    # 4) Attribute mutation on edge-entities
    # ------------------------------------------------------------
    attr_items = [
        (
            eid,
            {
                "weight": float(i % 5),
                "label": f"type_{i % 3}",
            },
        )
        for i, eid in enumerate(ee_ids)
    ]

    with measure() as m_attr:
        mem_attr = _measure_mem(
            lambda: G.add_edge_entities_bulk(attr_items, slice="edge_edge_entities")
        )

    results["edge_entity_attr_update"] = {
        "count": len(attr_items),
        "wall_time_s": m_attr["wall_time_s"],
        "bytes": mem_attr,
        "bytes_per_item": mem_attr / max(1, len(attr_items)),
    }

    # ------------------------------------------------------------
    # Final counters
    # ------------------------------------------------------------
    results["final_counts"] = {
        "vertices": G.number_of_vertices(),
        "edges_total": G.number_of_edges(),
        "edge_entities": sum(
            1 for t in G.entity_types.values() if t == "edge"
        ),
    }

    return results


In [1]:
# benchmarks/core/expressiveness.py
import sys
import os
sys.path.insert(0, os.path.abspath(".."))
from annnet.core.graph import AnnNet
import random
import tempfile
from annnet.core.graph import AnnNet
from benchmarks.harness.metrics import measure


def run(scale):
    G = AnnNet(directed=True)

    with measure() as m_vertices:
        G.add_vertices_bulk(
            ({"vertex_id": f"v{i}"} for i in range(scale.vertices)),
            slice="base",
        )

    with measure() as m_edges:
        G.add_edges_bulk(
            {
                "source": f"v{i % scale.vertices}",
                "target": f"v{(i * 37) % scale.vertices}",
                "weight": float(i % 7),
                "edge_type": "regular",
            }
            for i in range(scale.edges))

    with tempfile.TemporaryDirectory() as tmp:
        path = f"{tmp}/graph.annnet"

        with measure() as m_write:
            G.write(path, overwrite=True)

        with measure() as m_read:
            _ = AnnNet.read(path)
    
    return {
        "vertices": m_vertices,
        "edges": m_edges,
        "total_vertices": G.number_of_vertices(),
        "total_edges": G.number_of_edges(),
        "write_annnet": m_write,
        "read_annnet": m_read,
    }


In [2]:
# benchmarks/core/operations.py
import random
from annnet.core.graph import AnnNet
from benchmarks.harness.metrics import measure


def run(scale):
    G = AnnNet(directed=True)
    
    # Build base graph
    vertices = [f"v{i}" for i in range(scale.vertices)]
    G.add_vertices_bulk(
        ({"vertex_id": vid} for vid in vertices),
        slice="base",
    )
    
    edges = []
    for i in range(scale.edges):
        edges.append({
            "source": f"v{i % scale.vertices}",
            "target": f"v{(i * 37) % scale.vertices}",
            "weight": float(i % 7),
            "edge_type": "regular",
        })
    G.add_edges_bulk(edges)
    
    hyperedges = []
    for _ in range(scale.hyperedges):
        hyperedges.append({
            "members": random.sample(vertices, min(5, len(vertices))),
            "weight": 1.0,
        })
    G.add_hyperedges_bulk(hyperedges)
    
    # Add slices
    for i in range(3):
        slice_name = f"slice_{i}"
        G.add_slice(slice_name)
        sample_edges = random.sample(list(G.edges()), min(scale.edges // 4, G.number_of_edges()))
        G.add_edges_to_slice_bulk(slice_name, sample_edges)
    
    # Benchmark operations
    
    with measure() as m_copy:
        G_copy = G.copy()
    
    with measure() as m_copy_history:
        G_copy_hist = G.copy(history=True)
    
    with measure() as m_subgraph_vertices:
        sample_v = random.sample(vertices, scale.vertices // 2)
        G_sub_v = G.subgraph(sample_v)
    
    with measure() as m_subgraph_edges:
        sample_e = random.sample(list(G.edges()), scale.edges // 2)
        G_sub_e = G.edge_subgraph(sample_e)
    
    with measure() as m_extract_subgraph:
        G_extract = G.extract_subgraph(
            vertices=sample_v,
            edges=sample_e
        )
    
    with measure() as m_reverse:
        G_rev = G.reverse()
    
    with measure() as m_subgraph_from_slice:
        G_slice = G.subgraph_from_slice("slice_0", resolve_slice_weights=True)
    
    with measure() as m_hash:
        h = hash(G)
    
    with measure() as m_memory_usage:
        mem = G.memory_usage()
    
    with measure() as m_vertex_incidence_sparse:
        M_sparse = G.vertex_incidence_matrix(values=True, sparse=True)
    
    with measure() as m_vertex_incidence_dense:
        M_dense = G.vertex_incidence_matrix(values=True, sparse=False)
    
    with measure() as m_vertex_incidence_lists:
        inc_lists = G.get_vertex_incidence_matrix_as_lists(values=False)
    
    return {
        "copy": m_copy,
        "copy_with_history": m_copy_history,
        "subgraph_vertices": m_subgraph_vertices,
        "subgraph_edges": m_subgraph_edges,
        "extract_subgraph": m_extract_subgraph,
        "reverse": m_reverse,
        "subgraph_from_slice": m_subgraph_from_slice,
        "hash": m_hash,
        "memory_usage": m_memory_usage,
        "vertex_incidence_sparse": m_vertex_incidence_sparse,
        "vertex_incidence_dense": m_vertex_incidence_dense,
        "vertex_incidence_lists": m_vertex_incidence_lists,
        "total_vertices": G.number_of_vertices(),
        "total_edges": G.number_of_edges(),
        "subgraph_v_vertices": G_sub_v.number_of_vertices(),
        "subgraph_v_edges": G_sub_v.number_of_edges(),
        "subgraph_e_vertices": G_sub_e.number_of_vertices(),
        "subgraph_e_edges": G_sub_e.number_of_edges(),
        "memory_bytes": mem,
        "graph_hash": h,
    }

In [1]:
# adapters
import sys
import os
sys.path.insert(0, os.path.abspath(".."))
from annnet.core.graph import AnnNet
import tempfile
from benchmarks.harness.metrics import measure
from annnet.adapters.networkx_adapter import to_nx, from_nx
from annnet.io.json_io import to_json, from_json
from annnet.adapters.pyg_adapter import to_pyg
from annnet.io.cx2_io import from_cx2, to_cx2
from annnet.io.SIF_io import to_sif, from_sif
from annnet.io.SBML_io import from_sbml
from annnet.io.dataframe_io import from_dataframes, to_dataframes
from annnet.io.GraphML_io import from_graphml, to_graphml, from_gexf, to_gexf
from annnet.io.GraphDir_Parquet_io import to_parquet_graphdir, from_parquet_graphdir
from annnet.adapters.graphtool_adapter import from_graphtool, to_graphtool
from annnet.adapters.igraph_adapter import from_igraph, to_igraph
from annnet.core.graph import AnnNet
import json


def run(scale):
    out = {}
    graph = AnnNet(directed=True)
    
    with measure() as m_vertices:
        graph.add_vertices_bulk(
            ({"vertex_id": f"v{i}"} for i in range(scale.vertices)),
            slice="base",
        )
    
    with measure() as m_edges:
        graph.add_edges_bulk(
            {
                "source": f"v{i % scale.vertices}",
                "target": f"v{(i * 37) % scale.vertices}",
                "weight": float(i % 7),
                "edge_type": "regular",
            }
            for i in range(scale.edges)
        )
    
    out["total_vertices"] = graph.number_of_vertices()
    out["total_edges"] = graph.number_of_edges()    
    out["vertices"] = m_vertices
    out["edges"] = m_edges
    
    # NetworkX adapter
    with measure() as m_nx_export:
        nxG, manifest = to_nx(graph)
    with measure() as m_nx_import:
        G2 = from_nx(nxG, manifest)
    out["nx_export"] = m_nx_export
    out["nx_import"] = m_nx_import
    out["nx_vertices"] = G2.number_of_vertices()
    out["nx_edges"] = G2.number_of_edges()
    
    # JSON adapter
    with tempfile.TemporaryDirectory() as td:
        path = os.path.join(td, "graph.json")
        with measure() as m_json_export:
            to_json(graph, path)
        size_bytes = os.path.getsize(path)
        with measure() as m_json_import:
            G3 = from_json(path)
    out["json_export"] = m_json_export
    out["json_import"] = m_json_import
    out["json_size_bytes"] = size_bytes
    out["json_vertices"] = G3.number_of_vertices()
    out["json_edges"] = G3.number_of_edges()
    
    # PyG (PyTorch Geometric) adapter
    with measure() as m_pyg_export:
        data = to_pyg(graph)
    tensor_bytes = sum(
        t.element_size() * t.nelement()
        for t in data.to_dict().values()
        if hasattr(t, "nelement")
    )
    out["pyg_export"] = m_pyg_export
    out["pyg_tensor_bytes"] = tensor_bytes
    
    # CX2 adapter
    with tempfile.TemporaryDirectory() as td:
        sk = to_cx2(graph)
        cx2_path = os.path.join(td, "graph.cx2")
        with measure() as m_cx2_export:
            with open(cx2_path, "w") as f:
                json.dump(sk, f)
        cx2_size = os.path.getsize(cx2_path)
        with measure() as m_cx2_import:
            G4 = from_cx2(cx2_path)
    out["cx2_export"] = m_cx2_export
    out["cx2_import"] = m_cx2_import
    out["cx2_size_bytes"] = cx2_size
    out["cx2_vertices"] = G4.number_of_vertices()
    out["cx2_edges"] = G4.number_of_edges()
    
    # SIF adapter
    with tempfile.TemporaryDirectory() as td:
        sif_path = os.path.join(td, "graph.sif")
        with measure() as m_sif_export:
            to_sif(graph, sif_path)
        sif_size = os.path.getsize(sif_path)
        with measure() as m_sif_import:
            G5 = from_sif(sif_path)
    out["sif_export"] = m_sif_export
    out["sif_import"] = m_sif_import
    out["sif_size_bytes"] = sif_size
    out["sif_vertices"] = G5.number_of_vertices()
    out["sif_edges"] = G5.number_of_edges()
    
    # DataFrame adapter
    with measure() as m_df_export:
        dfs = to_dataframes(graph)
    df_size = 0
    for df in dfs.values():
        if hasattr(df, 'memory_usage'):
            df_size += df.memory_usage(deep=True).sum()
        elif hasattr(df, 'estimated_size'):
            df_size += df.estimated_size()
    with measure() as m_df_import:
        G6 = from_dataframes(
            nodes=dfs.get('nodes'),
            edges=dfs.get('edges'),
            hyperedges=dfs.get('hyperedges'),
            slices=dfs.get('slices'),
            slice_weights=dfs.get('slice_weights')
        )
    out["df_export"] = m_df_export
    out["df_import"] = m_df_import
    out["df_memory_bytes"] = df_size
    out["df_vertices"] = G6.number_of_vertices()
    out["df_edges"] = G6.number_of_edges()
    
    # GraphML adapter
    with tempfile.TemporaryDirectory() as td:
        graphml_path = os.path.join(td, "graph.graphml")
        with measure() as m_graphml_export:
            to_graphml(graph, graphml_path)
        graphml_size = os.path.getsize(graphml_path)
        with measure() as m_graphml_import:
            G7 = from_graphml(graphml_path)
    out["graphml_export"] = m_graphml_export
    out["graphml_import"] = m_graphml_import
    out["graphml_size_bytes"] = graphml_size
    out["graphml_vertices"] = G7.number_of_vertices()
    out["graphml_edges"] = G7.number_of_edges()
    
    # GEXF adapter
    with tempfile.TemporaryDirectory() as td:
        gexf_path = os.path.join(td, "graph.gexf")
        with measure() as m_gexf_export:
            to_gexf(graph, gexf_path)
        gexf_size = os.path.getsize(gexf_path)
        with measure() as m_gexf_import:
            G8 = from_gexf(gexf_path)
    out["gexf_export"] = m_gexf_export
    out["gexf_import"] = m_gexf_import
    out["gexf_size_bytes"] = gexf_size
    out["gexf_vertices"] = G8.number_of_vertices()
    out["gexf_edges"] = G8.number_of_edges()
    
    # Parquet GraphDir adapter
    with tempfile.TemporaryDirectory() as td:
        parquet_dir = os.path.join(td, "graph_parquet")
        with measure() as m_parquet_export:
            to_parquet_graphdir(graph, parquet_dir)
        parquet_size = sum(
            os.path.getsize(os.path.join(parquet_dir, f))
            for f in os.listdir(parquet_dir)
            if os.path.isfile(os.path.join(parquet_dir, f))
        )
        with measure() as m_parquet_import:
            G9 = from_parquet_graphdir(parquet_dir)
    out["parquet_export"] = m_parquet_export
    out["parquet_import"] = m_parquet_import
    out["parquet_size_bytes"] = parquet_size
    out["parquet_vertices"] = G9.number_of_vertices()
    out["parquet_edges"] = G9.number_of_edges()
    
    # graph-tool adapter
    try:
        with measure() as m_gt_export:
            gt_graph, gt_manifest = to_graphtool(graph)
        with measure() as m_gt_import:
            G10 = from_graphtool(gt_graph, gt_manifest)
        out["graphtool_export"] = m_gt_export
        out["graphtool_import"] = m_gt_import
        out["graphtool_vertices"] = G10.number_of_vertices()
        out["graphtool_edges"] = G10.number_of_edges()
    except Exception as e:
        out["graphtool_error"] = str(e)
    
    # igraph adapter
    try:
        with measure() as m_ig_export:
            ig_graph, ig_manifest = to_igraph(graph)
        with measure() as m_ig_import:
            G11 = from_igraph(ig_graph, ig_manifest)
        out["igraph_export"] = m_ig_export
        out["igraph_import"] = m_ig_import
        out["igraph_vertices"] = G11.number_of_vertices()
        out["igraph_edges"] = G11.number_of_edges()
    except Exception as e:
        out["igraph_error"] = str(e)
    
    return out

/home/l1boll/miniconda3/envs/myenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
# each adapter
import sys
import os
sys.path.insert(0, os.path.abspath(".."))
from annnet.core.graph import AnnNet
import tempfile
from benchmarks.harness.metrics import measure
from annnet.adapters.networkx_adapter import to_nx, from_nx
from annnet.io.json_io import to_json, from_json
from annnet.adapters.pyg_adapter import to_pyg
from annnet.io.cx2_io import from_cx2, to_cx2
from annnet.io.SIF_io import to_sif, from_sif
from annnet.io.SBML_io import from_sbml
from annnet.io.dataframe_io import from_dataframes, to_dataframes
from annnet.io.GraphML_io import from_graphml, to_graphml, from_gexf, to_gexf
from annnet.io.GraphDir_Parquet_io import to_parquet_graphdir, from_parquet_graphdir
from annnet.adapters.graphtool_adapter import from_graphtool, to_graphtool
from annnet.adapters.igraph_adapter import from_igraph, to_igraph
from annnet.core.graph import AnnNet
import json
import random


def run(scale):
    out = {}
    graph = AnnNet(directed=True)
    
    with measure() as m_vertices:
        graph.add_vertices_bulk(
            ({"vertex_id": f"v{i}"} for i in range(scale.vertices)),
            slice="base",
        )
    
    with measure() as m_edges:
        graph.add_edges_bulk(
            {
                "source": f"v{i % scale.vertices}",
                "target": f"v{(i * 37) % scale.vertices}",
                "weight": float(i % 7),
                "edge_type": "regular",
            }
            for i in range(scale.edges)
        )
    vertices = [f"v{i}" for i in range(scale.vertices)]
    hyperedges = []
    for _ in range(scale.hyperedges):
        hyperedges.append({
            "members": random.sample(vertices, min(5, len(vertices))),
            "weight": 1.0,
        })
    
    with measure() as m_he:
        graph.add_hyperedges_bulk(hyperedges)

    out["total_vertices"] = graph.number_of_vertices()
    out["total_edges"] = graph.number_of_edges()
    out["total_hyper_edges"] = scale.hyperedges
    out["vertices"] = m_vertices
    out["edges"] = m_edges
    out["hyperedges"] = m_he
    
    # NetworkX adapter
    with measure() as m_nx_export:
        nxG, manifest = to_nx(graph, hyperedge_mode= "expand")
    with measure() as m_nx_import:
        G2 = from_nx(nxG, manifest)
    out["nx_export"] = m_nx_export
    out["nx_import"] = m_nx_import
    out["nx_vertices"] = G2.number_of_vertices()
    out["nx_edges"] = G2.number_of_edges()
    
    # JSON adapter
    """with tempfile.TemporaryDirectory() as td:
        path = os.path.join(td, "graph.json")
        with measure() as m_json_export:
            to_json(graph, path)
        size_bytes = os.path.getsize(path)
        with measure() as m_json_import:
            G3 = from_json(path)
    out["json_export"] = m_json_export
    out["json_import"] = m_json_import
    out["json_size_bytes"] = size_bytes
    out["json_vertices"] = G3.number_of_vertices()
    out["json_edges"] = G3.number_of_edges()
    """
    # PyG (PyTorch Geometric) adapter
    """with measure() as m_pyg_export:
        data = to_pyg(graph)
    tensor_bytes = sum(
        t.element_size() * t.nelement()
        for t in data.to_dict().values()
        if hasattr(t, "nelement")
    )
    out["pyg_export"] = m_pyg_export
    out["pyg_tensor_bytes"] = tensor_bytes
    """
    # CX2 adapter
    """with tempfile.TemporaryDirectory() as td:
        sk = to_cx2(graph)
        cx2_path = os.path.join(td, "graph.cx2")
        with measure() as m_cx2_export:
            with open(cx2_path, "w") as f:
                json.dump(sk, f)
        cx2_size = os.path.getsize(cx2_path)
        with measure() as m_cx2_import:
            G4 = from_cx2(cx2_path)
    out["cx2_export"] = m_cx2_export
    out["cx2_import"] = m_cx2_import
    out["cx2_size_bytes"] = cx2_size
    out["cx2_vertices"] = G4.number_of_vertices()
    out["cx2_edges"] = G4.number_of_edges()
    """
    # SIF adapter
    """with tempfile.TemporaryDirectory() as td:
        sif_path = os.path.join(td, "graph.sif")
        with measure() as m_sif_export:
            to_sif(graph, sif_path)
        sif_size = os.path.getsize(sif_path)
        with measure() as m_sif_import:
            G5 = from_sif(sif_path)
    out["sif_export"] = m_sif_export
    out["sif_import"] = m_sif_import
    out["sif_size_bytes"] = sif_size
    out["sif_vertices"] = G5.number_of_vertices()
    out["sif_edges"] = G5.number_of_edges()
    """
    # DataFrame adapter
    """with measure() as m_df_export:
        dfs = to_dataframes(graph)
    df_size = 0
    for df in dfs.values():
        if hasattr(df, 'memory_usage'):
            df_size += df.memory_usage(deep=True).sum()
        elif hasattr(df, 'estimated_size'):
            df_size += df.estimated_size()
    with measure() as m_df_import:
        G6 = from_dataframes(
            nodes=dfs.get('nodes'),
            edges=dfs.get('edges'),
            hyperedges=dfs.get('hyperedges'),
            slices=dfs.get('slices'),
            slice_weights=dfs.get('slice_weights')
        )
    out["df_export"] = m_df_export
    out["df_import"] = m_df_import
    out["df_memory_bytes"] = df_size
    out["df_vertices"] = G6.number_of_vertices()
    out["df_edges"] = G6.number_of_edges()
    """
    # GraphML adapter
    """with tempfile.TemporaryDirectory() as td:
        graphml_path = os.path.join(td, "graph.graphml")
        with measure() as m_graphml_export:
            to_graphml(graph, graphml_path)
        graphml_size = os.path.getsize(graphml_path)
        with measure() as m_graphml_import:
            G7 = from_graphml(graphml_path)
    out["graphml_export"] = m_graphml_export
    out["graphml_import"] = m_graphml_import
    out["graphml_size_bytes"] = graphml_size
    out["graphml_vertices"] = G7.number_of_vertices()
    out["graphml_edges"] = G7.number_of_edges()
    """
    # GEXF adapter
    """with tempfile.TemporaryDirectory() as td:
        gexf_path = os.path.join(td, "graph.gexf")
        with measure() as m_gexf_export:
            to_gexf(graph, gexf_path)
        gexf_size = os.path.getsize(gexf_path)
        with measure() as m_gexf_import:
            G8 = from_gexf(gexf_path)
    out["gexf_export"] = m_gexf_export
    out["gexf_import"] = m_gexf_import
    out["gexf_size_bytes"] = gexf_size
    out["gexf_vertices"] = G8.number_of_vertices()
    out["gexf_edges"] = G8.number_of_edges()
    """
    # Parquet GraphDir adapter
    """with tempfile.TemporaryDirectory() as td:
        parquet_dir = os.path.join(td, "graph_parquet")
        with measure() as m_parquet_export:
            to_parquet_graphdir(graph, parquet_dir)
        parquet_size = sum(
            os.path.getsize(os.path.join(parquet_dir, f))
            for f in os.listdir(parquet_dir)
            if os.path.isfile(os.path.join(parquet_dir, f))
        )
        with measure() as m_parquet_import:
            G9 = from_parquet_graphdir(parquet_dir)
    out["parquet_export"] = m_parquet_export
    out["parquet_import"] = m_parquet_import
    out["parquet_size_bytes"] = parquet_size
    out["parquet_vertices"] = G9.number_of_vertices()
    out["parquet_edges"] = G9.number_of_edges()
    """
    # graph-tool adapter
    """try:
        with measure() as m_gt_export:
            gt_graph, gt_manifest = to_graphtool(graph)
        with measure() as m_gt_import:
            G10 = from_graphtool(gt_graph, gt_manifest)
        out["graphtool_export"] = m_gt_export
        out["graphtool_import"] = m_gt_import
        out["graphtool_vertices"] = G10.number_of_vertices()
        out["graphtool_edges"] = G10.number_of_edges()
    except Exception as e:
        out["graphtool_error"] = str(e)
    """
    # igraph adapter
    """try:
        with measure() as m_ig_export:
            ig_graph, ig_manifest = to_igraph(graph)
        with measure() as m_ig_import:
            G11 = from_igraph(ig_graph, ig_manifest)
        out["igraph_export"] = m_ig_export
        out["igraph_import"] = m_ig_import
        out["igraph_vertices"] = G11.number_of_vertices()
        out["igraph_edges"] = G11.number_of_edges()
    except Exception as e:
        out["igraph_error"] = str(e)
    """
    return out

/home/l1boll/miniconda3/envs/myenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class scale():
    vertices = 10000
    edges = 10000
    slices = 3
    hyperedges = 10000

In [3]:
run(scale)

{'total_vertices': 10000,
 'total_edges': 20000,
 'total_hyper_edges': 10000,
 'vertices': {'wall_time_s': 0.036541426001349464,
  'rss_before_mb': 996.14453125,
  'rss_after_mb': 1006.01953125,
  'rss_delta_mb': 9.875},
 'edges': {'wall_time_s': 0.1681860550015699,
  'rss_before_mb': 1006.01953125,
  'rss_after_mb': 1014.39453125,
  'rss_delta_mb': 8.375},
 'hyperedges': {'wall_time_s': 0.5542241750008543,
  'rss_before_mb': 1015.39453125,
  'rss_after_mb': 1041.01953125,
  'rss_delta_mb': 25.625},
 'nx_export': {'wall_time_s': 0.7821945069990761,
  'rss_before_mb': 1041.01953125,
  'rss_after_mb': 1139.89453125,
  'rss_delta_mb': 98.875},
 'nx_import': {'wall_time_s': 1.96360769599778,
  'rss_before_mb': 1139.89453125,
  'rss_after_mb': 1342.03515625,
  'rss_delta_mb': 202.140625},
 'nx_vertices': 10000,
 'nx_edges': 20000}

In [4]:
import cProfile
import pstats
from annnet.adapters.networkx_adapter import to_nx

graph = AnnNet()
graph.add_vertices_bulk(
    ({"vertex_id": f"v{i}"} for i in range(scale.vertices)),
    slice="base",
)
    
graph.add_edges_bulk(
    {
        "source": f"v{i % scale.vertices}",
        "target": f"v{(i * 37) % scale.vertices}",
        "weight": float(i % 7),
        "edge_type": "regular",
    }
    for i in range(scale.edges)
)
vertices = [f"v{i}" for i in range(scale.vertices)]
hyperedges = []
for _ in range(scale.hyperedges):
    hyperedges.append({
        "members": random.sample(vertices, min(5, len(vertices))),
        "weight": 1.0,
    })
graph.add_hyperedges_bulk(hyperedges)

pr = cProfile.Profile()
pr.enable()
nxG, manifest = to_nx(graph, hyperedge_mode="reify")
pr.disable()

stats = pstats.Stats(pr)
stats.sort_stats('cumulative')
stats.print_stats(30)


         1740165 function calls in 1.125 seconds

   Ordered by: cumulative time
   List reduced from 83 to 30 due to restriction <30>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    1.124    0.562 /home/l1boll/miniconda3/envs/myenv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3665(run_code)
        2    0.000    0.000    1.124    0.562 {built-in method builtins.exec}
        1    0.011    0.011    1.124    1.124 /tmp/ipykernel_4446/3848574810.py:1(<module>)
        1    0.410    0.410    1.113    1.113 /mnt/c/Users/pc/desktop/anananan/annnet/adapters/networkx_adapter.py:247(to_nx)
   110000    0.349    0.000    0.396    0.000 /home/l1boll/miniconda3/envs/myenv/lib/python3.11/site-packages/networkx/classes/multidigraph.py:428(add_edge)
        1    0.050    0.050    0.139    0.139 /mnt/c/Users/pc/desktop/anananan/annnet/adapters/networkx_adapter.py:73(_export_legacy)
    20000    0.048    0.000    0.071    0.000 /

In [3]:
g  = AnnNet()

In [4]:
g.add_vertex("a", a = 6)


'a'

In [10]:
g.add_edge("a", "b", edge_id= "edge_0", a = 45)

'edge_0'

In [8]:
g.vertex_attributes

vertex_id,a
str,f64
"""a""",6.0
"""b""",8.9


In [7]:
g.set_vertex_attrs("b", a = 8.9)

In [13]:
g.edge_attributes


edge_id,a
str,f64
"""edge_0""",45.0
"""edge_1""",4.5


In [12]:
g.add_edge("a", "c", a = 4.5)

'edge_1'